# Bibliotecas

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
import ee
import numpy as np
from osgeo import ogr
from osgeo import gdal
import glob,pygeoj
import rasterio
from rasterio.plot import show
from patchify import patchify, unpatchify


import json
import math
import logging
import time
from datetime import datetime, date, timedelta

import folium
from PIL import Image
from matplotlib import pyplot as plt
from IPython.display import Image

logging.getLogger('googleapicliet.discovery_cache').setLevel(logging.ERROR)

gpu_dict    = {'4090':{'GPU_AFFINTY' : 0, 'GPU_MEMORY_LIMIT_GB':12}}
sel_gpu     = '4090'
GPU_AFFINTY = gpu_dict[sel_gpu]['GPU_AFFINTY'] 
GPU_MEMORY_LIMIT_GB = gpu_dict[sel_gpu]['GPU_MEMORY_LIMIT_GB']
USER_EE_PROJECT='USER_PROJECT_ID'

try:
    ee.Initialize(project = USER_EE_PROJECT)
except:
    ee.Authenticate()
    ee.Initialize(project = USER_EE_PROJECT)

In [ ]:
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Tensorflow Version:',tf.__version__)
print('Folium Version:',folium.__version__)

gpus = tf.config.list_physical_devices('GPU')

if gpus:
  try:
    tf.config.set_visible_devices(gpus[GPU_AFFINTY], 'GPU')
    GPU_MEMORY_LIMIT_GB = GPU_MEMORY_LIMIT_GB * 1e3
    if GPU_MEMORY_LIMIT_GB == 0:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    else:
        tf.config.set_logical_device_configuration(gpus[GPU_AFFINTY],[tf.config.LogicalDeviceConfiguration(memory_limit=GPU_MEMORY_LIMIT_GB)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)

Tensorflow Version: 2.15.0
Folium Version: 0.14.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs


In [ ]:
def create_directory(new_folder):
  ''' Check if any of the specified folder already exist'''
  if not os.path.exists(new_folder):
      print(f'lets make the directory: {new_folder}')
      os.makedirs(new_folder)
  else: return

  def check_file_exists(paths):
    """Check if any of the specified paths already exist"""
    for path in paths:
        if os.path.exists(path):
            return True
    return False

# ENV Configs

In [ ]:
# General 
BIOMA      = 'pampa'
BIOMA_DICT = {'amazonia':{'BIOMA_CODE':1,'DATASET_FLAG':'amz'},
              'caatinga':{'BIOMA_CODE':2, 'DATASET_FLAG':'caatinga'},
              'cerrado':{'BIOMA_CODE':3, 'DATASET_FLAG':'cerr'},
              'mataatlantica':{'BIOMA_CODE':4, 'DATASET_FLAG':'mata'},
              'pampa':{'BIOMA_CODE':5, 'DATASET_FLAG':'pampa', },
              'pantanal':{'BIOMA_CODE':6, 'DATASET_FLAG':'pantanal'}}

BIOMA_CODE = BIOMA_DICT[BIOMA]['BIOMA_CODE']
BIOME_DATASET_FLAG = BIOMA_DICT[BIOMA]['DATASET_FLAG']

VERSION        = '1'
MOSAIC_VERSION = '1'

BUCKET = 'mb10-mining'
GDRIVE = 'mb10-unet-mineracao-'+BIOMA
FOLDER = 'training_samples'
TRAINING_BASE = 'training_patches_'+MOSAIC_VERSION
EVAL_BASE     = 'eval_patches_'+MOSAIC_VERSION

#Local paths
LOCAL_PATH  = '~/mb10-unet-mineracao/'+BIOMA
MODEL_DIR   = LOCAL_PATH+'/checkpoint/v'+VERSION
NFS_PATH = '~/mb10-unet-mineracao/'+BIOMA
OUTPUT_PATH = NFS_PATH+'/output/v'+VERSION

create_directory(MODEL_DIR)
create_directory(OUTPUT_PATH)

# Exportation Configs
BUCKET_patch = BUCKET
FOLDER_patch = 'allPatch'
FOLDER_classification = 'mb10_mining_'+VERSION

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands   = ['green','red','nir','swir1','NDVI','MNDWI']

BANDS    = opticalBands
RESPONSE = 'supervised'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE  = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 0
EVAL_SIZE = 0

# Specify model training parameters.
BATCH_SIZE  = 16
DROPOUT     = 0.3
EPOCHS      = 50
BUFFER_SIZE = 1000
OPTIMIZER   = 'Nadam' 
LOSS        = 'BinaryCrossentropy'
METRICS     = ['BinaryIoU']

In [ ]:
mosaic_year   = 2022

supervisedImg = ee.Image('projects/mapbiomas-workspace/COLECAO9/mineracao/'+str(mosaic_year)+'-5').gte(1).unmask(0).rename(RESPONSE)
supervisedChannel = supervisedImg.toByte().rename(RESPONSE);

image = ee.Image('projects/mapbiomas-workspace/TRANSVERSAIS/ZONACOSTEIRA6/mosaic_'+str(mosaic_year)).addBands(supervisedChannel)

mapid = image.getMapId({'bands': ['swir1', 'nir', 'red'], 'min': 30, 'max': 150})
map = folium.Map(location=[-5.9442, -56.5265])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Planet',
    overlay=True,
    name='Mosaic composite',
  ).add_to(map)
mapid = supervisedChannel.select(RESPONSE).getMapId({'min': 0, 'max': 1, 'pallete':'#ff0000'})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='supervisedLayer',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
featureStack = ee.Image.cat([
  image.select(BANDS).unmask(0),
  image.select(RESPONSE).unmask(0)
]).float()
    
list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)

In [ ]:
trainingPolys =  ee.FeatureCollection.loadBigQueryTable('solved-mb10.mb10_database.amostras_'+str(BIOME_DATASET_FLAG)+'_treino','geo')
evalPolys = ee.FeatureCollection.loadBigQueryTable('solved-mb10.mb10_database.amostras_'+str(BIOME_DATASET_FLAG)+'_teste','geo')
polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[-1.3621, -45.2738], zoom_start=5)

map.add_child(folium.LayerControl())
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map

In [ ]:
TRAINING_BASE= 'training_patches_'+str(VERSION)+'_'+BIOMA
EVAL_BASE = 'eval_patches_'+str(VERSION)+'_'+BIOMA
FOLDER_TRAIN = 'training_samples_v'+str(VERSION)+'_MB10_'+BIOMA
FOLDER_EVAL = 'eval_samples_v'+str(VERSION)+'_MB10_'+BIOMA

# Train/Test Chips Exportation

In [ ]:
# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())
# These numbers determined experimentally.
n = 20 # Number of shards in each polygon.
N = 200 # Total sample size in each polygon.

#Add some generalism
TRAIN_SIZE = trainingPolys.size().getInfo()*N
EVAL_SIZE = evalPolys.size().getInfo()*N
print('TRAIN:'+str(TRAIN_SIZE))
print('EVAL:'+str(EVAL_SIZE))

# Export all the training data (in many pieces), with one task 
# per geometry.
for g in range(trainingPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(trainingPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n, # Size of the shard.
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = TRAINING_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc, 
    folder = GDRIVE+'_'+FOLDER_TRAIN, 
    fileNamePrefix = desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()

# Export all the evaluation data.
for g in range(evalPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(evalPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n,
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = EVAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc, 
    folder = GDRIVE+'_'+FOLDER_EVAL, 
    fileNamePrefix = desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE],
  )
  task.start()

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns: 
    A dictionary of tensors, keyed by feature name.
  """
  print(FEATURES_DICT)
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns: 
    A dtuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns: 
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

In [ ]:
from tensorflow.keras import preprocessing
import random

TRAIN_SIZE = 0

def augment_spatial(image, label):
    """Randomly translates/pads the image with a small tilt effect."""
    padded_image = tf.pad(image, [[64, 64], [64, 64], [0, 0]], mode='CONSTANT') 
    padded_label = tf.pad(label, [[64, 64], [64, 64], [0, 0]], mode='CONSTANT')
    random_x = random.randint(0, 128)
    random_y = random.randint(0, 128)
    cropped_image = tf.slice(padded_image, [random_x,random_y, 0], [256, 256, 6])
    cropped_label = tf.slice(padded_label, [random_x, random_y, 0], [256, 256, 1])
    return cropped_image,cropped_label


def get_training_dataset():
    TRAINING_BASE = "training_patches_1"
    glob = '~/train/*.tfrecord.gz'
    dataset = get_dataset(glob)
    TRAIN_SIZE = dataset.reduce(np.int64(0), lambda x,_ : x + 1).numpy()
    augmented_dataset = dataset.map(augment_spatial, num_parallel_calls=100)
    dataset = augmented_dataset.concatenate(dataset).shuffle(20000, reshuffle_each_iteration=True).batch(BATCH_SIZE).repeat().prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

training = get_training_dataset()
first_element = next(iter(training))

In [ ]:
EVAL_SIZE = 0

def augment_spatial_eval(image, label):
    """Randomly translates/pads the image with a small tilt effect."""
    padded_image = tf.pad(image, [[64, 64], [64, 64], [0, 0]], mode='CONSTANT')
    padded_label = tf.pad(label, [[64, 64], [64, 64], [0, 0]], mode='CONSTANT')
    random_x = random.randint(0, 128)
    random_y = random.randint(0, 128)
    cropped_image = tf.slice(padded_image, [random_x,random_y, 0], [256, 256, 6])
    cropped_label = tf.slice(padded_label, [random_x, random_y, 0], [256, 256, 1])
    return cropped_image,cropped_label


def get_eval_dataset():
    EVAL_BASE = 'eval_patches_1'
    glob = '~/eval/*.tfrecord.gz'
    dataset = get_dataset(glob)
    EVAL_SIZE = dataset.reduce(np.int64(0), lambda x,_ : x + 1).numpy()
    augmented_dataset = dataset.map(augment_spatial, num_parallel_calls=100)
    dataset = augmented_dataset.concatenate(dataset).shuffle(6556*2, reshuffle_each_iteration=True).map(augment_spatial_eval, num_parallel_calls=100).batch(2).repeat()
    return dataset

evaluation = get_eval_dataset()

# UNET

In [ ]:

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers


def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = layers.Input(shape=[None, None, len(BANDS)])
	encoder0_pool, encoder0 = encoder_block(inputs, 64) 
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 128)
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 256)
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 512)
	center = conv_block(encoder3_pool, 1024)
	decoder4 = decoder_block(center, encoder3, 512)
	decoder3 = decoder_block(decoder4, encoder2, 256)
	decoder2 = decoder_block(decoder3, encoder1, 128)
	decoder1 = decoder_block(decoder2, encoder0, 64)

	dropout = layers.Dropout(DROPOUT, name="dropout", noise_shape=None, seed=None)(decoder1)
	outputs = layers.Conv2D(1, (1, 1),  activation=tf.nn.sigmoid, padding='same', kernel_initializer=tf.keras.initializers.GlorotNormal())(dropout)
	model = models.Model(inputs=[inputs], outputs=[outputs])
	
	optimizer = tf.keras.optimizers.Nadam( learning_rate=0.000005, name='optimizer')

	model.compile(
		optimizer=optimizer, 
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS]
    )

	return model

In [ ]:
m = get_model()
EPOCH = 0
CHECK_MODEL_DIR_MG = '~/checkpoint/v1/cp-00'+str(EPOCH)+'.keras'
m.load_weights(CHECK_MODEL_DIR_MG)

In [ ]:
def previewClass(epoch,log):
    """Callback funtion for training acompaniment by visual verification"""
    counter = 0
    for batch in evaluation.shuffle(1000).take(3):
        pureImage = batch[0]
        supervised = batch[1]
        stacked = tf.transpose(pureImage[0], [0, 1, 2]).numpy()
        stackedS = tf.transpose(supervised[0], [0, 1, 2]).numpy()

        test_pred_raw = m.predict(pureImage)
        test_pred_raw = tf.transpose(test_pred_raw[0],[0, 1, 2]).numpy()
        fig = plt.figure(figsize=[12,4])
        # show original image
        fig.add_subplot(131)
        plt.imshow(stacked[:,:,0:3].astype(np.uint8), interpolation='nearest', vmin=0, vmax=255)
        fig.add_subplot(132)
        plt.imshow(stackedS[:,:,0], interpolation='nearest',cmap="gray")
        fig.add_subplot(133)
        plt.imshow(test_pred_raw[:,:,0], interpolation='nearest',cmap="gray")
        plt.show()
        counter = counter+1

In [ ]:
plt.style.use("ggplot")
checkpoint_path = MODEL_DIR+"/cp-{epoch:04d}.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='output/v'+VERSION+'/log_model',write_images=True)

cp_callback  = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,verbose=1, save_weights_only=False,save_best_only=False,save_freq='epoch')
img_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=previewClass)


result = m.fit(x=training,
  epochs=100,
  initial_epoch=0, # REMEBER TO CHANGE THIS INITIAL EPOCH PARAM, WHEN OTHER MODEL HAS BEEN LOADED
  steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE), 
  verbose=1,
  shuffle=True,
  validation_data=evaluation,
  validation_steps=EVAL_SIZE,
  callbacks = [cp_callback,tensorboard]
)

# Grid

# Export mosaics to GDrive

In [ ]:
def doExport(out_image_base,year,region_id, kernel_buffer, roi):
  """Run the image export task."""
  image = ee.Image('projects/'+USER_EE_PROJECT+'/assets/USER_PATH/mosaic_'+str(year))
  # Export the image, specifying scale and region.
  task = ee.batch.Export.image.toDrive(
    image          = image.select(BANDS).toFloat(),
    description    = out_image_base+'_'+str(year),
    fileNamePrefix = out_image_base+'_'+str(year), 
    folder         = 'mosaics_landsat/'+str(year),
    scale          = 30,
    region         = roi,
    fileFormat     = 'GEOTIFF',
    formatOptions  = { 
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 157286400
    }
  )
  task.start()  


In [ ]:
# Run the export.
for region in full_aqua_regions:
    region_id = int(region.properties['id'])
    image_base_name =f'v{MOSAIC_VERSION}_L9_grid_{region_id}'
    if int(region_id):
      print('Region:',int(region_id))
      for y in range(1985, 2021): 
          doExport(image_base_name,y, kernel_buffer, region.geometry.coordinates[0])

# Prediction

In [2]:
def mosaic_predict(mosaic_lzw,year, region_id, output_path, version, kernel_dim, optical_bands, optical_indices, model, mosaic_scale):
    """Executes segmentation over mosaic data exported from EE as .geotiff

    Parameters
    ----------
    mosaic_lzw : geotiff
        Geotiff data representing the LANDSAT mosaic. The region exported must be the same as the region_id
    year : int
        The year of said geotiff
    region_id : int
        The id of the geojson grid that identifies the geotiff region
    output_path: str
        The output path for the segmented data
    version: int
        The segmentation version
    kernel_dim: int
        The dimention of the patches to be segmented
    optical_bands: list
        List of optical bands present on each geotiff
    optical_indices: list
        List of optical indices on each geotiff
    model: Tensorflow model
        The model trained
    mosaic_scale: int
        The scale of the geotiff. Tipically, for landsat the scale is 30

    Returns
    -------
    Geotiff
        A geotiff corresonding to the segmentation fo the target for the mosaic_lzw
    """
    paths_to_check = [

        f'{output_path}/{year}/{EPOCH}/outimage_v{version}_e{EPOCH}_grid_{region_id}_{year}_lzw.tif',
        f'{output_path}/{year}/{EPOCH}/outimage_v{version}_e{EPOCH}_grid_{region_id}_{year}_byte_lzw.tif',
        f'{output_path}/{year}/{EPOCH}/outimage_v{version}_e{EPOCH}_grid_{region_id}_{year}_byte_lzw2.tif',
    ]
    if check_file_exists(paths_to_check):
        return

    with rasterio.open(mosaic_lzw, 'r') as ds:
        arr = ds.read()
    arr = np.clip(arr, 0, None)
    
    img_arr_original = arr.astype(np.float32)
    if img_arr_original.shape[0] > 6:
        img_arr_original = img_arr_original[1:, :, :]
    img_arr_original = np.nan_to_num(img_arr_original, nan=0.0) 

    
    iit = np.array(np.transpose(img_arr_original,[1,2,0]))

    ''' MAKE THE IMAGE QUADRATIC '''
    arr_shape_xy = np.array(img_arr_original.shape[1:])
    min_dim = arr_shape_xy.min()
    index_min_dim = np.where(arr_shape_xy==min_dim)[0][0]
    if index_min_dim == 0: 
        img_arr = img_arr_original[:, :, :min_dim] 
    elif index_min_dim ==1:  
        img_arr = img_arr_original[:, :min_dim, :]
    
    ''' ENSURE IMAGE DIMENSIONS ARE MULTIPLES OF kernel_dim '''
    pad_size = (kernel_dim - (min_dim % kernel_dim)) % kernel_dim 
    
    if pad_size > 0:
        img_arr = np.pad(img_arr, ((0, 0), (0, pad_size), (0, pad_size)), mode='edge')

    ''' FULL BANDS (6) '''
    disired_dims    = kernel_dim*2 
    bands           = optical_bands + optical_indices
    patches         = patchify(img_arr, (len(bands), disired_dims, disired_dims), step=kernel_dim)
    dim             = patches.shape[1]
    patch2          = patches.reshape((1, dim**2, len(bands), disired_dims, disired_dims))
    patch2_reshaped = patch2[0].reshape((dim**2, len(bands), disired_dims, disired_dims))
    patch3          = np.transpose(patch2_reshaped, [0,2,3,1])

    curr_patch      = tf.data.Dataset.from_tensor_slices(patch3).batch(1)
    with tf.device('/device:GPU:0'):
        predictions_arr = model.predict(curr_patch, batch_size=8,steps=None, verbose=1)
    patchesPerRow  = dim 
    TotalPatches   = dim**2 
    patchDimension = [disired_dims,disired_dims] 

    counter       = 1
    rowCounter    = 1
    globalCounter = 0
    finalArray    = np.array([])
    rowArray      = np.array([])

    for raw_record in predictions_arr:
        raw_record = np.squeeze(raw_record) 
        rows,cols = raw_record.shape
        
        limite_esquerda = kernel_dim//2               
        limite_direita  = kernel_dim + (kernel_dim//2)
        limite_inferior = kernel_dim + (kernel_dim//2)
        limite_superior = kernel_dim//2               
        if rowCounter == 1: 
            limite_superior = 0 
        if (counter == 1) or (counter == patchesPerRow+1):
            limite_esquerda = 0
        if (counter == patchesPerRow+1) and rowCounter == 1:
            limite_superior = kernel_dim//2

        if counter == patchesPerRow:
            limite_direita = kernel_dim * 2

        if rowCounter == (TotalPatches/patchesPerRow) or (rowCounter == (TotalPatches/patchesPerRow)-1 and counter == patchesPerRow+1):
            limite_inferior = kernel_dim * 2

        raw_record = raw_record[limite_superior:limite_inferior,limite_esquerda:limite_direita]
        if rowCounter == 1:
            finalArray = rowArray
        if counter <= patchesPerRow: 
            if counter == 1:
                rowArray = raw_record
            else:
                rowArray = np.concatenate((rowArray,raw_record), axis = 1)
            counter = counter+1
        else:
            counter = 2
            rowCounter = rowCounter+1
            if np.array_equal(finalArray,rowArray):
                finalArray = rowArray
            else:
                finalArray = np.concatenate((finalArray,rowArray),axis=0)
            rowArray = raw_record
        globalCounter = globalCounter+1
    finalArray = np.concatenate((finalArray,rowArray),axis=0)
    finalArray_padded = dynamic_slice_or_pad(arr_shape_xy, finalArray)
    threshold      = 0.1
    rows,cols = finalArray_padded.shape
    finalArray_padded = np.array([finalArray_padded]) 
    output_path = f'{output_path}/{year}/{EPOCH}'
    create_directory(output_path)
    raster_uri     = output_path + '/UNET_v'+version+'grid'+str(region_id)+'_'+str(year)+'.tif'
    try:
        if not np.any(np.isnan(finalArray_padded)):
            finalArray_padded = np.round((finalArray_padded.astype(np.float32))*255).astype(np.uint8)
        raster_uri_lzw = f'{output_path}/outimage_v'+version+'_e'+str(EPOCH)+'_grid_'+str(region_id)+'_'+str(year)+'_byte_lzw.tif'
        data_type = "uint8"
    except Exception as inst:
        print(f'ERROR: For grid {region_id} \n {inst.args}, {inst}\n Raster in float')
        raster_uri_lzw = f'{output_path}/outimage_v'+version+'_e'+str(EPOCH)+'_grid_'+str(region_id)+'_'+str(year)+'_float_lzw.tif'
        data_type = "float32"  

    with rasterio.open(raster_uri,'w',
                  driver="GTiff",
                  height=rows,
                  width=cols,
                  count=1,
                  dtype=data_type,
                  crs='EPSG:4326',
                  transform=ds.transform,
                  nodata=0) as dataset:
                      dataset.write(finalArray_padded)
    dataset = gdal.Open(raster_uri, gdal.GA_Update)
    !gdal_translate -of GTiff -ot Byte -co "COMPRESS=LZW" -co "PREDICTOR=2" -co "TILED=YES" {raster_uri} {raster_uri_lzw} 
    !rm {raster_uri}

In [8]:
def dynamic_slice_or_pad(target_shape, predicted_img):

    """Add padding to the segmentation output so that it mayches the mosaic geotiff input dimentions

    Parameters
    ----------
    target_shape : list
        List with the size of total rows and columns of the input image
    predicted_img : Numpy array
        Array repreenting the segmented output

    Returns
    -------
    Numpy array
        Numpy array representing the segmentation output properly padded
    """
    target_rows, target_cols = target_shape
    pad_rows = target_rows - predicted_img.shape[0]
    pad_cols = target_cols - predicted_img.shape[1]

    if pad_rows < 0:
        predicted_img = predicted_img[:target_rows, :]
    elif pad_rows > 0:
        predicted_img = np.pad(predicted_img, ((0, pad_rows), (0, 0)), mode='constant')

    if pad_cols < 0:
        predicted_img = predicted_img[:, :target_cols]
    elif pad_cols > 0:
        predicted_img = np.pad(predicted_img, ((0, 0), (0, pad_cols)), mode='constant')
    
    return predicted_img

In [ ]:
# Prediction iteration
ROOT_PATH      = './'
YEAR           = 2024
MOSAIC_VERSION = '1'
mosaic_scale   = 30


GRIDS_IDS    = pygeoj.load(f'{ROOT_PATH}/GRIDS/GRID_ALLCLASSES_COLECAO_10_STACK_BIOMA_V1.geojson')
loaded_model = m
reduced_grid = [int(feature.properties['id']) for feature in GRIDS_IDS if feature.properties['mining'] == 1 and feature.properties['CD_Bioma'] == BIOMA_CODE ]
reduced_grid = [int(n + 1) for n in reduced_grid]
reduced_grid.sort()

start = time.time()
for EPOCH in range(70,80,10):
    print(EPOCH)
    CHECK_MODEL_DIR_MG = '~/'+str(BIOMA)+'/checkpoint/v1/cp-00'+str(EPOCH)+'.keras'
    m.load_weights(CHECK_MODEL_DIR_MG)
    for year in range(2020,2024):
        print(f'/n/nYEAR:{year}')
        i = 0

        for region_id in reduced_grid:
            print(f'REGION ID {region_id}')
            tf.keras.backend.clear_session()
            start = time.time()

            MOSAIC_PATH  = f'{ROOT_PATH}/mosaico_landsat/{year}'
            search_pattern = os.path.join(MOSAIC_PATH, f'v{MOSAIC_VERSION}_*_grid_{region_id}_{year}*.tif')
            matching_files = glob.glob(search_pattern)
            
            if matching_files:
                i = i+1
                region_mosaic_file = matching_files[0]
                mosaic_predict(region_mosaic_file, year, region_id, OUTPUT_PATH, VERSION, KERNEL_SIZE, opticalBands, [], loaded_model, mosaic_scale)
            else:
                print(i)
                print("No matching:",search_pattern)
    end = time.time()
    print('Prediction Time per year = '+str(end - start))